<a href="https://colab.research.google.com/github/pneuly/whisper-asr-colab/blob/main/whisper_asr_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import logging
import locale
from torch.cuda import empty_cache
from google.colab import files

locale.getpreferredencoding = lambda: "UTF-8"

# @title 自動文字起こし{ display-mode: "form" }
audio = 'https://www.youtube.com/watch?v=xAmEQOqtMvA'  # @param {type:"string"}
model_size = "large-v3-turbo" # @param ["large-v3-turbo", "large-v3", "large-v2", "large", "medium", "small", "base", "tiny"] {allow-input: true}
diarization = True  # @param {type:"boolean"}
password = ""  # @param {type:"string"}
start_time = ""  # @param {type:"string"}
end_time = ""  # @param {type:"string"}
timestamp_offset = ""  # @param {type:"string"}
skip_silence = True  # @param {type:"boolean"}
hotwords = "次に、これです。" # @param {type:"string"}
batch_size = 1 # @param {type:"number"}
realtime = False  # @param {type:"boolean"}
HUGGING_FACE_TOKEN = ""
initial_prompt = "定刻になりましたので、開始いたします。"
prefix = None
vad_filter = False

# @markdown ###<br/><b>〔設定の説明〕</b>
# @markdown <b>audiopath:</b> 文字起こしする音声ファイルの場所<br/>
# @markdown 　　Youtubeの場合： https://www.youtube.com/......<br/>
# @markdown 　　手動で音声をアップロードした場合： 230401_1010.mp3 など<br/>
# @markdown <br/><b>model_size:</b> 音声認識のモデルサイズ
# @markdown <br/><b>diarization:</b> 発言者別の文字起こしファイルを作成するか
# @markdown #### <br/><b><font color= "blue">以下は必要な場合のみ設定</font></b>
# @markdown <b>password:</b> パスワードを指定（Webexなど）</b>
# @markdown <br/><b>start_time:</b> 開始時間 hh:mm:ss</b>（指定しない場合は最初から）
# @markdown <br/><b>end_time:</b> 終了時間 hh:mm:ss（指定しない場合は最後まで）
# @markdown <br/><b>timestamp_offset:</b> タイムスタンプを指定の時間だけずらす hh:mm:ss（指定しない場合はstart_timeと連動）
# @markdown <br/><b>skip_silence:</b> 先頭と最後の無音区間を削除
# @markdown <br/><b>hotwords:</b> キーワード（次に、これです。は句読点を付けるために入れています。）
# @markdown <br/><b>batch_size:</b> 2以上にすると少し早くなります。1の場合と2以上の場合で文字起こし結果が変わります。
# @markdown <br/><b>reatime: </b>ストリーミングをリアルタイムで文字起こしをする場合のみオンにしてください。</font>

# ----- main routine ------
logger = logging.getLogger(__name__)
logger.setLevel(logging.WARNING)
if not logger.hasHandlers():
    logger.addHandler(logging.StreamHandler())

if diarization and HUGGING_FACE_TOKEN == "":
    raise ValueError("HUGGING_FACE_TOKEN にトークンを入力してください")

if audio == "":
    audio = list(files.upload())[0]

!pip install git+https://github.com/pneuly/whisper-asr-colab.git@main -q
!pip install ctranslate2==4.4.0
from whisper_asr_colab.worker import Worker # noqa: E402
from whisper_asr_colab.audio import Audio  # noqa: E402

audio = Audio.from_path_or_url(audio)
if start_time:
    audio.start_time = start_time
if end_time:
    audio.end_time = end_time

worker = Worker(
    audio=audio,
    model_size=model_size,
    language="ja",
    diarization=diarization,
    password=password,
    initial_prompt=initial_prompt,
    realtime=realtime,
    batch_size=batch_size,
    hugging_face_token=HUGGING_FACE_TOKEN,
    hotwords=hotwords,
    prefix=prefix,
    vad_filter=False,
    skip_silence=skip_silence,
)
if timestamp_offset:
    worker.timestamp_offset = timestamp_offset

worker.run()

# gc GPU RAM
del worker
empty_cache()